# Cleaning: Cycle Share

There are 3 datasets that provide data on the stations, trips, and weather from 2014-2016.

**Station dataset**

* station_id: station ID number
* name: name of station
* lat: station latitude
* long: station longitude
* install_date: date that station was placed in service
* install_dockcount: number of docks at each station on the installation date
* modification_date: date that station was modified, resulting in a change in location or dock count
* current_dockcount: number of docks at each station on 8/31/2016
* decommission_date: date that station was placed out of service

**Trip dataset**

* trip_id: numeric ID of bike trip taken
* starttime: day and time trip started, in PST
* stoptime: day and time trip ended, in PST
* bikeid: ID attached to each bike
* tripduration: time of trip in seconds
* from_station_name: name of station where trip originated
* to_station_name: name of station where trip terminated
* from_station_id: ID of station where trip originated
* to_station_id: ID of station where trip terminated
* usertype: "Short-Term Pass Holder" is a rider who purchased a 24-Hour or 3-Day Pass; "Member" is a rider who purchased a Monthly or an Annual Membership
* gender: gender of rider
* birthyear: birth year of rider

**Weather dataset** contains daily weather information in the service area

## 1. Import all sets into a dictionary and correct any errors

In [3]:
import pandas as pd
import numpy as np
from pandas import Series
from pandas import DataFrame as DF

In [7]:
from os import listdir

In [11]:
files = listdir('cycle_share')
data = {}

for f in files:
    k = f.split('.')[0]
    path = 'cycle_share/' + f
    try:
        data[k] = pd.read_csv(path)
    except Exception as e:
        print('File : {}\n'.format(f))
        print(e)

File : trip.csv

Error tokenizing data. C error: Expected 12 fields in line 50794, saw 20



In [13]:
with open('cycle_share/trip.csv') as f:
    lines = f.readlines()
for l in lines[50793:50795]:
    print(l, '\n')

59000,"4/17/2015 14:21","4/17/2015 19:21","SEA00362",17990.668,"6th Ave S & S King St","Westlake Ave & 6th Ave","ID-04","SLU-15"trip_id","starttime","stoptime","bikeid","tripduration","from_station_name","to_station_name","from_station_id","to_station_id","usertype","gender","birthyear"
 

431,"10/13/2014 10:31","10/13/2014 10:48","SEA00298",985.935,"2nd Ave & Spring St","Occidental Park / Occidental Ave S & S Washington St","CBD-06","PS-04","Member","Male",1960
 



In [14]:
len(lines[50793].split(',')), len(lines[50794].split(','))

(20, 12)

In [15]:
lines[0].split(',')

['"trip_id"',
 '"starttime"',
 '"stoptime"',
 '"bikeid"',
 '"tripduration"',
 '"from_station_name"',
 '"to_station_name"',
 '"from_station_id"',
 '"to_station_id"',
 '"usertype"',
 '"gender"',
 '"birthyear"\n']

In [16]:
bad_line = lines[50793]
bad_toks = bad_line.split(',')

In [17]:
bad_toks

['59000',
 '"4/17/2015 14:21"',
 '"4/17/2015 19:21"',
 '"SEA00362"',
 '17990.668',
 '"6th Ave S & S King St"',
 '"Westlake Ave & 6th Ave"',
 '"ID-04"',
 '"SLU-15"trip_id"',
 '"starttime"',
 '"stoptime"',
 '"bikeid"',
 '"tripduration"',
 '"from_station_name"',
 '"to_station_name"',
 '"from_station_id"',
 '"to_station_id"',
 '"usertype"',
 '"gender"',
 '"birthyear"\n']

In [19]:
new_toks = bad_toks[:9]
new_toks

['59000',
 '"4/17/2015 14:21"',
 '"4/17/2015 19:21"',
 '"SEA00362"',
 '17990.668',
 '"6th Ave S & S King St"',
 '"Westlake Ave & 6th Ave"',
 '"ID-04"',
 '"SLU-15"trip_id"']

In [24]:
end = new_toks[-1]
end = '"' + end.split('"')[1] + '"'
end

'"SLU-15"'

In [30]:
new_toks[-1] = end
new_toks.append(2*',' + '\n')
new_line = ','.join(new_toks)
new_line

'59000,"4/17/2015 14:21","4/17/2015 19:21","SEA00362",17990.668,"6th Ave S & S King St","Westlake Ave & 6th Ave","ID-04","SLU-15","SLU-15","SLU-15",,,\n'

In [31]:
len(new_line.split(','))

14

In [32]:
lines[50793] = new_line

In [40]:
with open('cycle_share/trip_fixed.csv', 'w') as f:
    for l in lines:
        f.write(l)

In [67]:
from os import listdir

files = listdir('cycle_share')
data = {}
for f in files:
    if f != 'trip.csv':
        k = f.split('.')[0]  # remove .csv
        path = 'cycle_share/' + f
        try:
            if k == 'trip_fixed':
                k = 'trip'
            data[k] = pd.read_csv(path)
        except Exception as e:
            print('File : {}\n'.format(f))
            print(e)

File : trip_fixed.csv

Error tokenizing data. C error: Expected 12 fields in line 50794, saw 14



## 2. Print data summaries including the number of null values. Should we drop or try to correct any of the null values?

In [47]:
for k in data.keys():
    print(k.upper(), '\n')
    print('Null counts')
    print(data[k].isnull().sum(), '\n')
    print(data[k].describe().T, '\n')
    print(data[k].describe(include=['O']).T, '\n', 50*'-', '\n')

STATION 

Null counts
station_id            0
name                  0
lat                   0
long                  0
install_date          0
install_dockcount     0
modification_date    41
current_dockcount     0
decommission_date    54
dtype: int64 

                   count        mean       std         min         25%  \
lat                 58.0   47.624796  0.019066   47.598488   47.613239   
long                58.0 -122.327242  0.014957 -122.355230 -122.338735   
install_dockcount   58.0   17.586207  3.060985   12.000000   16.000000   
current_dockcount   58.0   16.517241  5.117021    0.000000   16.000000   

                          50%         75%         max  
lat                 47.618591   47.627712   47.666145  
long              -122.328207 -122.316691 -122.284119  
install_dockcount   18.000000   18.000000   30.000000  
current_dockcount   18.000000   18.000000   26.000000   

                  count unique                 top freq
station_id           58     58        

In [49]:
data['weather'].Mean_Temperature_F.fillna(method='ffill', inplace = True)

## 3. Create a column in the trip table that contains only the date (no time)

In [64]:
data.keys()

dict_keys(['station', 'weather'])

## 4. Merge weather data with trip data and be sure not to lose any trip data

In [132]:
data['trip']

KeyError: 'trip'

## 5. Drop records that are completely duplicated (all values). Check for and inspect any duplicate trip_id values that remain. Remove if they exist.

## 6. Create columns for lat & long values for the from- and to- stations

## 7. Write a function to round all `tripduration` values to the nearest half second increment and then round all the values in the data

## 8. Verify that `trip_duration` matches the timestamps to within 60 seconds

## 9.Something is wrong with the `Max_Gust_Speed_MPH` column. Identify and correct the problem, then save the data.

# Cleaning: Movies

This data set contains 28 attributes related to various movie titles that have been scraped from IMDb. The set is supposed to contain unique titles for each record, where each record has the following attributes:

"movie_title" "color" "num_critic_for_reviews" "movie_facebook_likes" "duration" "director_name" "director_facebook_likes" "actor_3_name" "actor_3_facebook_likes" "actor_2_name" "actor_2_facebook_likes" "actor_1_name" "actor_1_facebook_likes" "gross" "genres" "num_voted_users" "cast_total_facebook_likes" "facenumber_in_poster" "plot_keywords" "movie_imdb_link" "num_user_for_reviews" "language" "country" "content_rating" "budget" "title_year" "imdb_score" "aspect_ratio"

The original set is available kaggle ([here](https://www.kaggle.com/deepmatrix/imdb-5000-movie-dataset))

## 1. Check for and correct similar values in `color`, `language`,  and `country`

In [68]:
import pandas as pd
from pandas import Series
from pandas import DataFrame as DF
import numpy as np

In [70]:
movies = pd.read_csv('movies/movies_data.csv')

In [71]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

In [73]:
for col in ['color', 'language', 'country']:
    print(col, '/n')
    print(movies[col].value_counts().sort_index(), '\n\n')

color /n
Black and White     206
Color              4799
black and white       3
color                16
Name: color, dtype: int64 


language /n
Aboriginal       2
Arabic           5
Aramaic          1
Bosnian          1
Cantonese       11
Chinese          3
Czech            1
Danish           5
Dari             2
Dutch            4
Dzongkha         1
English       4704
Filipino         1
French          73
German          19
Greek            1
Hebrew           5
Hindi           28
Hungarian        1
Icelandic        2
Indonesian       2
Italian         11
Japanese        18
Kannada          1
Kazakh           1
Korean           8
Mandarin        26
Maya             1
Mongolian        1
None             2
Norwegian        4
Panjabi          1
Persian          4
Polish           4
Portuguese       8
Romanian         2
Russian         11
Slovenian        1
Spanish         40
Swahili          1
Swedish          5
Tamil            1
Telugu           1
Thai             3
Urdu             1

In [76]:
movies.loc[(movies.color == 'color'), 'color'] = 'Color'
movies.loc[(movies.color == 'black and white'), 'color'] = 'Black and White'
movies.color.value_counts()

Color              4815
Black and White     209
Name: color, dtype: int64

## 2. Create a function that detects and lists non-numeric columns containing values with leading or trailing whitespace. Remove the whitespace in these columns.

In [86]:
def has_white(data, cols):
    white_list = []
    for col in cols:
        for x in data[col]:
            try:
                l = x.split(' ')
                if (l[0] == '') | (l[-1] == ''):
                    print('{} has whitespace'.format(col))
                    white_list.append(col)
                    break
            except Exception:
                continue
    return white_list

In [87]:
str_cols = movies.select_dtypes(include = ['O']).columns
white_list = has_white(movies, str_cols)
white_list

director_name has whitespace
actor_2_name has whitespace
movie_title has whitespace


['director_name', 'actor_2_name', 'movie_title']

In [89]:
movies[white_list] = movies[white_list].apply(lambda x: x.str.strip())
has_white(movies, str_cols)

[]

## 3. Remove duplicate records. Inspect any remaining duplicate movie titles.

In [90]:
movies.shape

(5043, 28)

In [93]:
movies.drop_duplicates(inplace = True)
movies.shape

(4998, 28)

In [94]:
movies.movie_title.nunique(), len(movies)

(4916, 4998)

In [95]:
movies.movie_title.value_counts().describe()

count    4916.000000
mean        1.016680
std         0.132763
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         3.000000
Name: movie_title, dtype: float64

In [99]:
movie_counts = movies.movie_title.value_counts()
potential_dups = movie_counts[movie_counts > 1].index
potential_dups

Index(['Ben-Hur', 'King Kong', 'Home', 'Syriana', 'Lucky Number Slevin',
       'Oz the Great and Powerful', 'Creepshow', 'The Lovers', 'RoboCop',
       '20,000 Leagues Under the Sea', 'Pan', 'The Island', 'The Gift',
       'Glory', 'Aloha', 'Conan the Barbarian', 'The Host', 'Sabotage', 'Juno',
       'Halloween', 'Mercury Rising', 'Clash of the Titans', 'Day of the Dead',
       'Dredd', 'House of Wax', 'The Astronaut's Wife', 'The Karate Kid',
       'The Lovely Bones', 'Skyfall', 'Cinderella', 'Precious', 'Side Effects',
       'Chasing Liberty', 'The Return of the Living Dead', 'The Dead Zone',
       'The Fog', 'The Tourist', 'Point Break', 'Twilight',
       'Victor Frankenstein', 'The Watch', 'The Day the Earth Stood Still',
       'The Fast and the Furious', 'Out of the Blue',
       'Dodgeball: A True Underdog Story', 'Spider-Man 3',
       'Around the World in 80 Days', 'Jack Reacher', 'Unknown',
       'Murder by Numbers', 'Carrie', 'The Last House on the Left',
       'C

## 4. Create a function that returns two arrays: one for titles that are truly duplicated, and  one for duplicated titles are not the same movie.
* hint: do this by comparing the imdb link values

In [107]:
def movie_duplicates(movie, potential):
    subset = ['movie_title', 'movie_imdb_link']
    dup_mask = movies.duplicated(subset=subset)
    duplicated = movies.loc[dup_mask].movie_title.unique()
    not_duplicated = Series(potential)[~Series(potential).isin(duplicated)].values
    return duplicated, not_duplicated

In [108]:
dup, not_dup = movie_duplicates(movies, potential_dups)
not_dup

array(['The Host', 'The Dead Zone', 'Out of the Blue'], dtype=object)

## 5. Alter the names of duplicate titles that are different movies so each is unique. Then drop all duplicate rows based on movie title.

In [110]:
for m in not_dup:
    for n, idx in enumerate(movies[movies.movie_title == m].index):
        movies.loc[idx, 'movie_title'] = m + '_{}'.format(n)
        
movies.drop_duplicates(subset=['movie_title'], inplace=True)

In [111]:
movies.movie_title.value_counts().head()

The Core                        1
Love in the Time of Monsters    1
Florence Foster Jenkins         1
Albert Nobbs                    1
State of Play                   1
Name: movie_title, dtype: int64

## 6. Create a series that ranks actors by proportion of movies they have appeared in

In [113]:
a1 = DF(movies.actor_1_name.value_counts())
a2 = DF(movies.actor_2_name.value_counts())
a3 = DF(movies.actor_3_name.value_counts())

In [115]:
a_all = a1.merge(a2, how='outer', left_index = True, right_index=True)\
    .merge(a3, how='outer', left_index=True, right_index=True)

In [119]:
actor_counts = a_all.sum(axis=1)
actor_ranks = (actor_counts/len(movies)).sort_values(ascending=False)
actor_ranks.head()

Robert De Niro    0.010775
Morgan Freeman    0.008742
Bruce Willis      0.007725
Matt Damon        0.007522
Johnny Depp       0.007319
dtype: float64

## 7. Create a table that contains the first and last years each actor appeared, and their length of history. Then include columns for the actors proportion and total number of movies.
* length is number of years they have appeared in movies

In [122]:
g1 = movies.groupby('actor_1_name')
g2 = movies.groupby('actor_2_name')
g3 = movies.groupby('actor_3_name')

hists = {}
for i, g in enumerate([g1, g2, g3]):
    k = 'g{}'.format(i)
    hists[k] = g.apply(lambda x: Series({'last': x['title_year'].max(),
                                        'first': x['title_year'].min()}))
hists['g0'].head()

,first,last
actor_1_name,,
50 Cent,2005.0,2005.0
A.J. Buckley,2015.0,2015.0
Aaliyah,2002.0,2002.0
Aasif Mandvi,2008.0,2008.0
Abbie Cornish,2009.0,2012.0


In [126]:
history = hists['g0'].merge(hists['g1'], how='outer', left_index=True, right_index=True)\
    .merge(hists['g2'], how = 'outer', left_index=True, right_index=True)
actor_hist = history.apply(lambda r: Series({'first': r.min(),
                                            'last':r.max(),
                                            'years':r.max() - r.min()}),
                     \
                           ]]]'axis = 1).sort_values(by='years', ascending=False)
actor_hist.head()

,first,last,years
Laurence Olivier,1940.0,2004.0,64.0
Debbie Reynolds,1952.0,2012.0,60.0
Marlon Brando,1951.0,2006.0,55.0
Dean Stockwell,1947.0,2001.0,54.0
Robert Duvall,1962.0,2014.0,52.0


In [127]:
actor_hist['movie_prop'] = actor_ranks
actor_hist['movie_count'] = round(actor_ranks*len(movies))
actor_hist.head()

,first,last,years,movie_prop,movie_count
Laurence Olivier,1940.0,2004.0,64.0,0.001016,5.0
Debbie Reynolds,1952.0,2012.0,60.0,0.000813,4.0
Marlon Brando,1951.0,2006.0,55.0,0.001830,9.0
Dean Stockwell,1947.0,2001.0,54.0,0.001016,5.0
Robert Duvall,1962.0,2014.0,52.0,0.004879,24.0


## 8. Create a column that gives each movie an integer ranking based on gross sales
* 1 should indicate the highest gross
* If more than one movie has equal sales, assign all the lowest rank in the group
* The next rank after this group should increase only by 1

In [128]:
df = DF({'a': [1,2,3,1,2,4]})
dense_method = df.rank(ascending=False, method='dense')
min_method = df.rank(ascending=False, method='min')
print(dense_method)
print(min_method)

     a
0  4.0
1  3.0
2  2.0
3  4.0
4  3.0
5  1.0
     a
0  5.0
1  3.0
2  2.0
3  5.0
4  3.0
5  1.0


In [130]:
movies['movie_sales_rank'] = movies.gross.rank(method='min', ascending=False)

In [131]:
movies[['gross', 'movie_sales_rank']].sort_values(by='gross', ascending=False).head()

,gross,movie_sales_rank
0,760505847.0,1.0
26,658672302.0,2.0
29,652177271.0,3.0
17,623279547.0,4.0
66,533316061.0,5.0
